In [13]:
from pydantic.main import Model

from agent import Agent
from tetris.game import Game
import numpy as np
import nest_asyncio
from tqdm import tqdm
nest_asyncio.apply()

# Evolutionary Algorithm
Fitness:
- Score weights based on average final Tetris score over N random games
Selection:
- Use fitness proportional selection to select top K performing options
Reproduction:
- Parent A contributes 1-4 random genes and Parent B contributes the compliment
Mutation:
- Apply m operations in sequence
  - No-op
  - Swap genes (i, j) 
  - Add/subtract value from gene i
  - Double/half value of gene i   

In [132]:
from pydantic import BaseModel, ConfigDict
from typing import List, Tuple, Any, Awaitable, Callable

import concurrent.futures
import asyncio

class EvoAlgo(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    selection_rate: int = 0.2
    mutation_rate: float = 0.2
    crossover_rate: float = 0.2
    increment_vals: List[float] = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0]
    multiply_vals: List[float] = [0.5, 0.75, 1.33, 2]
    weight_constraint: Tuple[float, float] = (0, 10)
    population: List[Agent] = []
    best_weights: List[np.ndarray] = []
    best_scores: List[float] = []
    seed: int = 87
    rng: np.random.Generator = np.random.default_rng(seed)
    
    def minimize(self, objective: Callable, num_generations: int = 10, population_size: int = 50) -> Tuple[List[np.ndarray], List[float], List[Agent]]:
        weights = self.rng.random(size=(population_size, 5)) * self.weight_constraint[1]
        agents = [Agent(weights=w) for w in weights]
        for g in range(num_generations):
            scores = []
            if g > 0:
                # Start new generation
                keep_agents = self.select(agents, scores)
                child_agents = self.crossover(keep_agents)
                mutated_agents = self.mutate(child_agents)
                agents = mutated_agents
            with concurrent.futures.ProcessPoolExecutor() as executor:
                scores = executor.map(objective, agents)
            scores = list(scores)
            self.best_scores.append(np.max(scores))
            self.best_weights.append(agents[np.argmax(scores)].weights)
            print(f"Best score was {self.best_scores[-1]} with weights {self.best_weights[-1]}")
            
        return self.best_weights, self.best_scores, agents
    
    def select(self, agents: List[Agent], scores: List[float]) -> List[Agent]:
        """Stochastic uniform sampling"""
        fps_scores = np.array(scores) / sum(scores)
        order = np.argsort(fps_scores)
        fps_scores = fps_scores[order]
        agents = [agents[i] for i in order]
        
        roulette_bins = np.cumsum(fps_scores)
        
        # Generate evenly spaced pointers
        N = len(agents)
        pointers = np.arange(N - 1) / (N - 1)
        offset = np.random.rand()  # Spin the roulette wheel!
        pointers = pointers + offset
        pointers[pointers > 1] = pointers[pointers > 1] - 1
        # Use digitize to select items
        keep_inds = np.digitize(pointers, roulette_bins)
        return [agents[i] for i in keep_inds]
    
    def crossover(self, agents: List[Agent]) -> List[Agent]:
        def _crossover_fn(a: Agent, b: Agent) -> Agent:
            genes = self.rng.permutation(5)
            selection = self.rng.choice(np.arange(4) + 1)
            new_weight = np.zeros(5)
            # Genetic selection should be complementary
            a_genes = genes[:selection]
            b_genes = genes[selection:]
            new_weight[a_genes] = a.weights[a_genes]
            new_weight[b_genes] = b.weights[b_genes]
            return Agent(weights=new_weight)
        
        new_agents = []
        for agent in agents:
            if self.rng.random() < self.crossover_rate:
                agent_coparent = self.rng.choice(agents)
                new_agents.append(_crossover_fn(agent, agent_coparent))
            else:
                new_agents.append(agent)
        return new_agents
    
    def _increment_gene(self, agent: Agent) -> Agent:
        i = self.rng.choice(5)
        val = self.rng.choice([-1, 1]) * self.rng.choice(self.increment_vals)
        agent.weights[i] += val
        agent.weights = np.clip(agent.weights, self.weight_constraint[0], self.weight_constraint[1])
        return agent
    
    def _multiply_gene(self, agent: Agent) -> Agent:
        i = self.rng.choice(5)
        val = self.rng.choice(self.multiply_vals)
        agent.weights[i] *= val
        agent.weights = np.clip(agent.weights, self.weight_constraint[0], self.weight_constraint[1])
        return agent
    
    def _swap_gene(self, agent: Agent) -> Agent:
        i, j = self.rng.choice(5, size=2, replace=False)
        agent.weights[[j, i]] = agent.weights[[i, j]]
        return agent
    
    def mutate(self, agents: List[Agent]) -> List[Agent]:
        mutation_fns = [self._swap_gene, self._increment_gene, self._multiply_gene]
        mutation_fn_weights = [0.2, 0.6, 0.2]
        mutated_agents = []
        for agent in agents:
            if self.rng.random() < self.mutation_rate:
                mutation_fn = self.rng.choice(mutation_fns, p=mutation_fn_weights)
                mutated_agents.append(mutation_fn(agent))
            else:
                mutated_agents.append(agent)
        # Ensure the best agent has at least one candidate
        mutated_agents[-1] = Agent(weights=self.best_weights[-1])
        return mutated_agents

In [133]:
GAME_SEED = 87
async def objective(agent: Agent) -> float:
    game = Game(agent, seed=GAME_SEED)
    results = []
    async for item in game.run():
        results.append(item)
    return game.score

In [134]:
algo = EvoAlgo(mutation_rate=0.8)

In [135]:
def process_agent(agent: Agent) -> float:
    return asyncio.run(objective(agent))


In [136]:
algo.minimize(process_agent, num_generations=10, population_size=100)

Best score was 41800 with weights [7.16486926 3.70265966 1.0339912  0.43048801 0.65572238]
Best score was 41800 with weights [6.93881597 1.96453073 9.88761146 0.34983371 1.65335652]
Best score was 41800 with weights [1.38435367 9.47622884 0.         5.37111807 1.26766473]
Best score was 41800 with weights [7.16486926 0.2854934  4.80265966 1.86097601 0.65572238]
Best score was 41800 with weights [0.01       2.47453073 9.99       3.28940798 1.65335652]
Best score was 41800 with weights [ 2.91706346  2.47453073 10.          3.01054717  1.65335652]
Best score was 41800 with weights [ 1.73470399  1.95453073 10.          0.93048801  1.49572238]
Best score was 44000 with weights [1.73470399 1.86097601 9.99       0.         0.64572238]
Best score was 41800 with weights [ 2.90706346  1.95453073 10.          1.88097601  1.49572238]
Best score was 49400 with weights [1.66097601 0.1527467  4.80265966 0.         0.65572238]


([array([ 7.16486926,  3.70265966,  0.43048801,  1.0439912 , -0.34427762]),
  array([13.87763194,  1.96453073,  9.68761146,  0.34983371,  1.65335652]),
  array([1.38435367, 9.47622884, 1.26766473, 5.36111807, 0.        ]),
  array([7.16486926, 0.2854934 , 9.60531933, 1.86097601, 0.45572238]),
  array([0.01      , 2.47453073, 7.4925    , 3.23940798, 1.65335652]),
  array([ 2.41706346,  2.47453073, 10.        ,  3.01054717,  1.55335652]),
  array([ 1.73470399,  1.95453073, 10.        ,  0.93048801,  1.49572238]),
  array([1.66097601, 1.73470399, 9.99      , 0.64572238, 0.        ]),
  array([ 2.95706346,  1.95453073, 10.        ,  1.88097601,  1.50572238]),
  array([1.66097601, 0.1527467 , 4.80265966, 0.        , 0.66572238])],
 [41800, 41800, 41800, 41800, 41800, 41800, 41800, 44000, 41800, 49400])

In [139]:
GAME_SEEDS = [87, 42, 101]
async def objective(agent: Agent, seed) -> float:
    game = Game(agent, seed=seed)
    results = []
    async for item in game.run():
        results.append(item)
    return game.score

def process_agent(agent: Agent) -> float:
    scores = []
    for seed in GAME_SEEDS:
        scores.append(asyncio.run(objective(agent, seed)))
    return np.mean(scores)


In [140]:
algo = EvoAlgo(mutation_rate=0.8, crossover_rate=0.4)

In [141]:
algo.minimize(process_agent, num_generations=10, population_size=20)

Best score was 38450.0 with weights [0.74980881 5.30095162 8.04020742 3.71680968 2.39185946]
Best score was 38450.0 with weights [0.74980881 2.76761358 8.54020742 3.71680968 2.02725388]
Best score was 38450.0 with weights [2.31305573 6.27873695 6.07579239 3.71680968 2.39185946]
Best score was 38450.0 with weights [0.84980881 6.27873695 8.04020742 3.81680968 2.39185946]
Best score was 38450.0 with weights [0.84980881 6.27873695 8.04020742 3.81680968 2.44185946]
Best score was 38450.0 with weights [2.31305573 6.27873695 8.04020742 3.79680968 2.44185946]
Best score was 38450.0 with weights [2.31305573 6.29873695 7.99020742 3.79680968 2.44185946]
Best score was 38450.0 with weights [0.79980881 6.27873695 8.04020742 3.71680968 2.44185946]
Best score was 38450.0 with weights [2.31305573 6.29873695 7.99020742 2.69680968 2.44185946]
Best score was 38450.0 with weights [2.31305573 6.29873695 6.99020742 2.69680968 2.44185946]


([array([3.73680968, 5.32095162, 8.04020742, 0.74980881, 2.39185946]),
  array([0.74980881, 2.76761358, 8.64020742, 3.71680968, 1.82725388]),
  array([2.31305573, 3.71680968, 6.07579239, 6.27873695, 1.59058654]),
  array([0.84980881, 6.27873695, 8.04020742, 3.81680968, 3.44185946]),
  array([0.79980881, 6.27873695, 8.04020742, 3.81680968, 2.44185946]),
  array([2.31305573, 6.29873695, 8.04020742, 3.79680968, 2.44185946]),
  array([2.26305573, 6.29873695, 7.99020742, 2.79680968, 2.44185946]),
  array([0.79980881, 6.26873695, 8.04020742, 3.71680968, 2.44185946]),
  array([2.31305573, 2.44185946, 7.99020742, 2.69680968, 6.29873695]),
  array([2.31305573, 6.29873695, 6.99020742, 2.69680968, 2.44185946])],
 [38450.0,
  38450.0,
  38450.0,
  38450.0,
  38450.0,
  38450.0,
  38450.0,
  38450.0,
  38450.0,
  38450.0])

In [ ]:
import concurrent.futures
import asyncio

num_trials = 10
game_seed = 42

async def objective(agent: Agent) -> float:
    game = Game(agent, seed=game_seed)
    async for _ in game.run():
        continue
    return game.score

def process_agent(agent: Agent) -> float:
    return asyncio.run(objective(agent))


agents = [Agent(np.random.randn(5)) for _ in range(num_trials)]
with concurrent.futures.ProcessPoolExecutor() as executor:
    scores = executor.map(process_agent, agents)
scores = list(scores)